pip install ultralytics

pip install xmltodict

In [ ]:
# import os
# import xmltodict


# def convert_coco_to_yolo(coco_xml_dir, yolo_txt_dir, class_mapping):
#     for filename in os.listdir(coco_xml_dir):
#         if filename.endswith(".xml"):
#             with open(os.path.join(coco_xml_dir, filename), 'r') as file:
#                 data = xmltodict.parse(file.read())

#             img_width = int(data['annotation']['size']['width'])
#             img_height = int(data['annotation']['size']['height'])

#             yolo_txt_path = os.path.join(yolo_txt_dir, filename.replace(".xml", ".txt"))

#             with open(yolo_txt_path, 'w') as file:
#                 for obj in data['annotation']['object']:
#                     class_name = obj['name']
#                     class_id = class_mapping.get(class_name)
                    
#                     if class_id is not None:
#                         x_min = int(obj['bndbox']['xmin'])
#                         y_min = int(obj['bndbox']['ymin'])
#                         x_max = int(obj['bndbox']['xmax'])
#                         y_max = int(obj['bndbox']['ymax'])

#                         x_center = (x_min + x_max) / 2 / img_width
#                         y_center = (y_min + y_max) / 2 / img_height
#                         width = (x_max - x_min) / img_width
#                         height = (y_max - y_min) / img_height

#                         file.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

# # Example usage
# class_mapping = {
#     'Freight': 0,
#     # Add more class mappings as needed
# }

# coco_xml_dir = '/Users/rishabhshah/Desktop/AIPI590/Project2/DatasetDetectron2/annotations'
# yolo_txt_dir = '/Users/rishabhshah/Desktop/AIPI590/Project2/DatasetDetectron2/annotations/outputYOLO.json'

# convert_coco_to_yolo(coco_xml_dir, yolo_txt_dir, class_mapping)

In [ ]:
# pip install pylabel

In [ ]:
import os 
import zipfile
from pylabel import importer


#Specify path to the coco.json file
path_to_annotations = "/Users/rishabhshah/Desktop/AIPI590/Project2/DatasetDetectron2/annotations/output.json"
#Specify the path to the images (if they are in a different folder than the annotations)
path_to_images = "/Users/rishabhshah/Desktop/AIPI590/Project2/MergeDataset/train"

#Import the dataset into the pylable schema 
dataset = importer.ImportCoco(path_to_annotations, path_to_images=path_to_images, name="BCCD_coco")
dataset.df.head(5)

In [ ]:
print(f"Number of images: {dataset.analyze.num_images}")
print(f"Number of classes: {dataset.analyze.num_classes}")
print(f"Classes:{dataset.analyze.classes}")
print(f"Class counts:\n{dataset.analyze.class_counts}")

In [ ]:
# https://github.com/pylabel-project/samples/blob/main/dataset_splitting.ipynb

dataset.splitter.GroupShuffleSplit(train_pct=.6, val_pct=.2, test_pct=.2)
dataset.analyze.ShowClassSplits()

dataset.export.ExportToYoloV5(output_path='/Users/rishabhshah/Desktop/AIPI590/Project2/trainingsplit/labels',yaml_file='/Users/rishabhshah/Desktop/AIPI590/Project2/trainingsplit/dataset.yaml', copy_images=True, use_splits=True)


In [ ]:
# https://github.com/pylabel-project/samples/blob/main/coco2yolov5.ipynb
dataset.path_to_annotations = "data/yolo"
dataset.export.ExportToYoloV5()[0]

In [ ]:
import os

from ultralytics import YOLO

ROOT_DIR = '/Users/rishabhshah/Desktop/AIPI590/Project2/trainingsplit'
# ROOT_DIR = '/Users/rishabhshah/Desktop/AIPI590/Project2/training'


# Load a model
# model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO('yolov8n.pt')

# Use the model
results = model.train(data=os.path.join(ROOT_DIR, "dataset.yaml"), epochs=1)  # train the model 